In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
def warn (*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [3]:
import pandas as pd
import requests

try:
   
    response = requests.get('https://api.thaimissing.go.th/api/v1/cir-Datacatalog/DataNorelativePerson')
    response.raise_for_status() 


    data = response.json()
    df = pd.DataFrame(data)  

    # แสดงข้อมูล 5 แถวแรก
    print(df.head())

except requests.exceptions.RequestException as e:
    print(f"เกิดข้อผิดพลาด: {e}")


         condition  autopsyYear gender         autopsyDate   causeofDeath  \
0  ศพมีสภาพสมบูรณ์       2568.0    ชาย  01 กุมภาพันธ์ 2568  ไม่ทราบสาเหตุ   
1  ศพมีสภาพสมบูรณ์       2568.0    ชาย  01 กุมภาพันธ์ 2568  ไม่ทราบสาเหตุ   
2  ศพมีสภาพสมบูรณ์       2568.0    ชาย      24 มกราคม 2568  ไม่ทราบสาเหตุ   
3  ศพมีสภาพสมบูรณ์       2568.0    ชาย      24 มกราคม 2568  ไม่ทราบสาเหตุ   
4  ศพมีสภาพสมบูรณ์       2568.0    ชาย      13 มกราคม 2568  ไม่ทราบสาเหตุ   

  foreign  
0    None  
1    None  
2    None  
3    None  
4    None  


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   condition     583 non-null    object 
 1   autopsyYear   582 non-null    float64
 2   gender        583 non-null    object 
 3   autopsyDate   583 non-null    object 
 4   causeofDeath  582 non-null    object 
 5   foreign       43 non-null     object 
dtypes: float64(1), object(5)
memory usage: 27.5+ KB


In [5]:
df.isnull().sum()

condition         0
autopsyYear       1
gender            0
autopsyDate       0
causeofDeath      1
foreign         540
dtype: int64

In [6]:
df.isnull().mean()

condition       0.000000
autopsyYear     0.001715
gender          0.000000
autopsyDate     0.000000
causeofDeath    0.001715
foreign         0.926244
dtype: float64

In [7]:
df['causeofDeath'].unique()

array(['ไม่ทราบสาเหตุ', None, 'อื่นๆ', 'อุบัติเหตุ', 'ปอดอักเสบติดเชื้อ',
       'อุบัติเหตุรักษาตัว', 'วัณโรคปอด', 'สันนิษฐานขาดอากาศจากการจมน้ำ',
       'โรคมะเร็ง', 'ลอยน้ำ', 'แขวนคอ', 'อุบัติเหตุจราจร', 'จมน้ำ',
       'ถูกทำร้ายร่างกาย', 'ระบบหัวใจล้มเหลว', 'ติดเชื้อในกระแสเลือด',
       'ผูกคอ', 'ถูกทำร้ายด้วยอาวุธมีด', 'ไม่ทราบสาเหตุ ',
       'ไม่ทราบสาเหตุ (ถูกทำร้ายร่างกาย)', 'ไฟฟ้าช็อต', 'ตกที่สูง'],
      dtype=object)

In [8]:
df.shape

(583, 6)

In [9]:
column_mapping = {
    'ไม่ทราบสาเหตุ': 'Unknown_Cause',
    'อื่นๆ': 'Other',
    'อุบัติเหตุ': 'Accident',
    'ปอดอักเสบติดเชื้อ': 'Pneumonia_Infection',
    'อุบัติเหตุรักษาตัว': 'Accident_Under_Treatment',
    'วัณโรคปอด': 'Pulmonary_Tuberculosis',
    'สันนิษฐานขาดอากาศจากการจมน้ำ': 'Presumed_Drowning',
    'โรคมะเร็ง': 'Cancer',
    'ลอยน้ำ': 'Floating_in_Water',
    'แขวนคอ': 'Hanging',
    'อุบัติเหตุจราจร': 'Traffic_Accident',
    'จมน้ำ': 'Drowning',
    'ถูกทำร้ายร่างกาย': 'Physical_Assault',
    'ระบบหัวใจล้มเหลว': 'Heart_Failure',
    'ติดเชื้อในกระแสเลือด': 'Sepsis',
    'ผูกคอ': 'Hanging',
    'ถูกทำร้ายด้วยอาวุธมีด': 'Assault_Stab_Wound',
    'ไม่ทราบสาเหตุ ': 'Unknown_Cause',
    'ไม่ทราบสาเหตุ (ถูกทำร้ายร่างกาย)': 'Unknown_Cause_(Assault)',
    'ไฟฟ้าช็อต': 'Electrocution',
    'ตกที่สูง': 'Fall_from_Height'
}

df_cleaned = df.replace(column_mapping)

In [10]:
df_cleaned.isnull().sum()

condition         0
autopsyYear       1
gender            0
autopsyDate       0
causeofDeath      1
foreign         540
dtype: int64

In [11]:
df_cleaned = df_cleaned.drop(columns=['foreign'], axis=1)


In [12]:
df_cleaned = df_cleaned.dropna()

In [13]:
df_final = df_cleaned[df_cleaned['condition'] != '-']


In [14]:
df_final

,condition,autopsyYear,gender,autopsyDate,causeofDeath
0,ศพมีสภาพสมบูรณ์,2568.0,ชาย,01 กุมภาพันธ์ 2568,Unknown_Cause
1,ศพมีสภาพสมบูรณ์,2568.0,ชาย,01 กุมภาพันธ์ 2568,Unknown_Cause
2,ศพมีสภาพสมบูรณ์,2568.0,ชาย,24 มกราคม 2568,Unknown_Cause
3,ศพมีสภาพสมบูรณ์,2568.0,ชาย,24 มกราคม 2568,Unknown_Cause
4,ศพมีสภาพสมบูรณ์,2568.0,ชาย,13 มกราคม 2568,Unknown_Cause
...,...,...,...,...,...
577,ศพมีสภาพสมบูรณ์,2562.0,ชาย,12 มกราคม 2562,Unknown_Cause
578,ศพมีสภาพสมบูรณ์,2562.0,ชาย,07 มกราคม 2562,Unknown_Cause
579,ศพมีสภาพสมบูรณ์,2562.0,ชาย,07 มกราคม 2562,Unknown_Cause
580,ศพมีสภาพสมบูรณ์,2562.0,ชาย,05 มกราคม 2562,Unknown_Cause


In [15]:
for col in df_final.columns:
    unique_values = df_final[col].unique()
    print(f"Column '{col}' has {len(unique_values)} unique values: {unique_values[:10]} ...")  # แสดงแค่ 10 ค่าแรก


Column 'condition' has 3 unique values: ['ศพมีสภาพสมบูรณ์' 'ศพมีสภาพเน่า' 'ศพมีสภาพเป็นกระดูก'] ...
Column 'autopsyYear' has 7 unique values: [2568. 2567. 2566. 2565. 2564. 2563. 2562.] ...
Column 'gender' has 2 unique values: ['ชาย' 'หญิง'] ...
Column 'autopsyDate' has 483 unique values: ['01 กุมภาพันธ์ 2568' '24 มกราคม 2568' '13 มกราคม 2568' '27 ธันวาคม 2567'
 '26 ธันวาคม 2567' '22 ธันวาคม 2567' '25 พฤศจิกายน 2567'
 '24 พฤศจิกายน 2567' '22 พฤศจิกายน 2567' '15 พฤศจิกายน 2567'] ...
Column 'causeofDeath' has 19 unique values: ['Unknown_Cause' 'Other' 'Accident' 'Accident_Under_Treatment'
 'Pulmonary_Tuberculosis' 'Presumed_Drowning' 'Cancer' 'Floating_in_Water'
 'Hanging' 'Traffic_Accident'] ...


In [16]:
df_final['condition'] = df_final['condition'].replace({
    'ศพมีสภาพสมบูรณ์': 'The body is intact',
    'ศพมีสภาพเน่า': 'The body is decomposed',
    'ศพมีสภาพเป็นกระดูก': 'The body is skeletal',
  
})



In [17]:
df_final['gender'] =  df_final['gender'].replace('ชาย','male')
df_final['gender'] =  df_final['gender'].replace('หญิง','female')

In [18]:
df_final['causeofDeath'].unique()

array(['Unknown_Cause', 'Other', 'Accident', 'Accident_Under_Treatment',
       'Pulmonary_Tuberculosis', 'Presumed_Drowning', 'Cancer',
       'Floating_in_Water', 'Hanging', 'Traffic_Accident', 'Drowning',
       'Physical_Assault', 'Heart_Failure', 'Sepsis',
       'Pneumonia_Infection', 'Assault_Stab_Wound',
       'Unknown_Cause_(Assault)', 'Electrocution', 'Fall_from_Height'],
      dtype=object)

In [19]:

df_final["autopsyDate"] = df_final["autopsyDate"]\
    .str.replace("มกราคม", "01", regex=False)\
    .str.replace("กุมภาพันธ์", "02", regex=False)\
    .str.replace("มีนาคม", "03", regex=False)\
    .str.replace("เมษายน", "04", regex=False)\
    .str.replace("พฤษภาคม", "05", regex=False)\
    .str.replace("มิถุนายน", "06", regex=False)\
    .str.replace("กรกฎาคม", "07", regex=False)\
    .str.replace("สิงหาคม", "08", regex=False)\
    .str.replace("กันยายน", "09", regex=False)\
    .str.replace("ตุลาคม", "10", regex=False)\
    .str.replace("พฤศจิกายน", "11", regex=False)\
    .str.replace("ธันวาคม", "12", regex=False)


df_final["autopsyDate"] = df_final["autopsyDate"].str.replace(r'(\d{4})', lambda x: str(int(x.group(0)) - 543), regex=True)

df_final["autopsyDate"] = pd.to_datetime(df_final["autopsyDate"], format="%d %m %Y").dt.date

print(df_final)



print(df_final)


              condition  autopsyYear gender autopsyDate   causeofDeath
0    The body is intact       2568.0   male  2025-02-01  Unknown_Cause
1    The body is intact       2568.0   male  2025-02-01  Unknown_Cause
2    The body is intact       2568.0   male  2025-01-24  Unknown_Cause
3    The body is intact       2568.0   male  2025-01-24  Unknown_Cause
4    The body is intact       2568.0   male  2025-01-13  Unknown_Cause
..                  ...          ...    ...         ...            ...
577  The body is intact       2562.0   male  2019-01-12  Unknown_Cause
578  The body is intact       2562.0   male  2019-01-07  Unknown_Cause
579  The body is intact       2562.0   male  2019-01-07  Unknown_Cause
580  The body is intact       2562.0   male  2019-01-05  Unknown_Cause
581  The body is intact       2562.0   male  2019-01-03  Unknown_Cause

[575 rows x 5 columns]
              condition  autopsyYear gender autopsyDate   causeofDeath
0    The body is intact       2568.0   male  2025-02-

In [20]:
df_final['autopsyYear'] = df_final['autopsyYear'].astype(int) - 543


In [21]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('panthep_test.db')
df_final.to_sql('people', conn, if_exists='replace', index=False)
conn.close()

In [22]:
df_final.to_excel('panthep_test.xlsx', index=False)

In [23]:
#import os


#file_path = '/kaggle/working/panthep_test.db'


#if os.path.exists(file_path):
    #os.remove(file_path)
    #print(f"ไฟล์ {file_path} ถูกลบเรียบร้อยแล้ว")
#else:
    #print(f"ไฟล์ {file_path} ไม่พบในระบบ")


In [24]:
df = pd.read_excel('/kaggle/working/panthep_test.xlsx',header=0)
df

,condition,autopsyYear,gender,autopsyDate,causeofDeath
0,The body is intact,2025,male,2025-02-01,Unknown_Cause
1,The body is intact,2025,male,2025-02-01,Unknown_Cause
2,The body is intact,2025,male,2025-01-24,Unknown_Cause
3,The body is intact,2025,male,2025-01-24,Unknown_Cause
4,The body is intact,2025,male,2025-01-13,Unknown_Cause
...,...,...,...,...,...
570,The body is intact,2019,male,2019-01-12,Unknown_Cause
571,The body is intact,2019,male,2019-01-07,Unknown_Cause
572,The body is intact,2019,male,2019-01-07,Unknown_Cause
573,The body is intact,2019,male,2019-01-05,Unknown_Cause


In [25]:
df['causeofDeath'].unique()

array(['Unknown_Cause', 'Other', 'Accident', 'Accident_Under_Treatment',
       'Pulmonary_Tuberculosis', 'Presumed_Drowning', 'Cancer',
       'Floating_in_Water', 'Hanging', 'Traffic_Accident', 'Drowning',
       'Physical_Assault', 'Heart_Failure', 'Sepsis',
       'Pneumonia_Infection', 'Assault_Stab_Wound',
       'Unknown_Cause_(Assault)', 'Electrocution', 'Fall_from_Height'],
      dtype=object)

In [26]:
import sqlite3
import pandas as pd

# สร้างการเชื่อมต่อกับฐานข้อมูล SQLite
conn = sqlite3.connect('/kaggle/working/panthep_test.db')

# อ่านข้อมูลจากตารางในฐานข้อมูล (เช่น `table_name`) มาเป็น DataFrame
df2 = pd.read_sql('SELECT * FROM people', conn)

# ปิดการเชื่อมต่อ
conn.close()

# แสดงผล DataFrame
print(df2)


              condition  autopsyYear gender autopsyDate   causeofDeath
0    The body is intact         2025   male  2025-02-01  Unknown_Cause
1    The body is intact         2025   male  2025-02-01  Unknown_Cause
2    The body is intact         2025   male  2025-01-24  Unknown_Cause
3    The body is intact         2025   male  2025-01-24  Unknown_Cause
4    The body is intact         2025   male  2025-01-13  Unknown_Cause
..                  ...          ...    ...         ...            ...
570  The body is intact         2019   male  2019-01-12  Unknown_Cause
571  The body is intact         2019   male  2019-01-07  Unknown_Cause
572  The body is intact         2019   male  2019-01-07  Unknown_Cause
573  The body is intact         2019   male  2019-01-05  Unknown_Cause
574  The body is intact         2019   male  2019-01-03  Unknown_Cause

[575 rows x 5 columns]


In [27]:
df2['causeofDeath'].unique()

array(['Unknown_Cause', 'Other', 'Accident', 'Accident_Under_Treatment',
       'Pulmonary_Tuberculosis', 'Presumed_Drowning', 'Cancer',
       'Floating_in_Water', 'Hanging', 'Traffic_Accident', 'Drowning',
       'Physical_Assault', 'Heart_Failure', 'Sepsis',
       'Pneumonia_Infection', 'Assault_Stab_Wound',
       'Unknown_Cause_(Assault)', 'Electrocution', 'Fall_from_Height'],
      dtype=object)

In [28]:
df2.isnull().sum()

condition       0
autopsyYear     0
gender          0
autopsyDate     0
causeofDeath    0
dtype: int64

In [29]:
df2.groupby('causeofDeath').size().reset_index(name='count')


,causeofDeath,count
0,Accident,4
1,Accident_Under_Treatment,2
2,Assault_Stab_Wound,1
3,Cancer,1
4,Drowning,3
5,Electrocution,1
6,Fall_from_Height,1
7,Floating_in_Water,5
8,Hanging,6
9,Heart_Failure,1
